In [2]:
import os
import glob
import json
import pickle
import torch
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm

from yolov3.modules import Darknet
from yolov3.coco import (
    load_classes, compute_coco_detections, compute_name_id_mappings,
    map_ids, convert_coco_detections, convert_coco_annotations
)
from yolov3.predictions import compute_predictions
from yolov3.data import (
    read_img, resize_aspect, img_to_tensor,
    inv_aspect_transform_results, clamp_results,
    aspect_transform
)
from yolov3.visualize import (
    uniform_color_palette,
    draw_predictions, draw_palette
)
from yolov3.bbox import bbox_iou
from yolov3.metrics import compute_interpolated_precision, bbox_match

In [3]:
coco_val_path = '/media/semyon/Data/Documents/coco/val2017'
coco_val_preds_path = '/media/semyon/Data/Documents/coco/preds_pt'

In [5]:
classes = load_classes('config/yolov3/coco.names')
num_classes = len(classes)

In [6]:
def compute_image_name(img_path):
    img_file_name = os.path.split(img_path)[1]
    img_id = os.path.splitext(img_file_name)[0]
    return img_id


def compute_output_path(img_path, output_dir):
    img_name = compute_image_name(img_path)
    result_file_name = f'{img_name}.pt'
    result_path = os.path.join(output_dir, result_file_name)
    return result_path

In [7]:
def json_load(file_path):
    with open(file_path, 'r') as input_file:
        result = json.load(input_file)
    return result


def json_dump(obj, file_path):
    with open(file_path, 'w') as output_file:
        json.dump(obj, output_file)

In [8]:
annotations_path = '/media/semyon/Data/Documents/coco/annotations/instances_val2017_nocrowd.json'
annotations = json_load(annotations_path)
coco_cat_ids_mapping_path = '/media/semyon/Data/Documents/coco/coco_cat_ids_mapping.json'
coco_cat_ids_mapping = json_load(coco_cat_ids_mapping_path)
coco_cat_ids_mapping = {int(k): v for k, v in coco_cat_ids_mapping.items()}
coco_cat_ids_mapping_inv = {v: k for k, v in coco_cat_ids_mapping.items()}
coco_bboxes = convert_coco_annotations(annotations, coco_cat_ids_mapping_inv)

In [9]:
yolo_detections_path = '/media/semyon/Data/Documents/coco/yolo_dt.json'
yolo_detections = json_load(yolo_detections_path)
yolo_bboxes = convert_coco_detections(yolo_detections, coco_cat_ids_mapping_inv)

In [10]:
name_id, id_name = compute_name_id_mappings(annotations['images'])

In [16]:
sum([len(v) for k, v in yolo_bboxes.items()])

30628

In [17]:
sum([len(v) for k, v in coco_bboxes.items()])

36335

In [18]:
n_preds = 100
iou_threshold = 0.75
area_range = [0 ** 2, 1e5 ** 2]
# area_range = [32 ** 2, 96 ** 2]
c_skip = 0
matches = []
matches_cat = defaultdict(list)
for image_id in id_name:
    if image_id in coco_bboxes:
        targets = coco_bboxes[image_id]
    else:
        targets = torch.empty(0, 6, dtype=torch.float32)
    if image_id in yolo_bboxes:
        preds = yolo_bboxes[image_id]
    else:
        preds = torch.empty(0, 7, dtype=torch.float32)
    if targets.numel() == 0 and preds.numel() == 0:
        c_skip += 1
        continue
    cats = torch.unique(torch.cat((preds[:, 5], targets[:, 4]))).detach().cpu().numpy()
    for c in cats:
        p = preds[preds[:, 5] == c]
        t = targets[targets[:, 4] == c]
        ious = bbox_iou(p[:, :4], t[:, :4])
        target_ids = t[:, 5].detach().cpu().numpy().astype(np.int64)
        match = bbox_match(
            p[:, :5], t[:, :4], ious,
            iou_threshold=iou_threshold,
            n_preds=n_preds,
            area_range=area_range
        )
        matches_cat[c - 1].append(match)
        pred_ids = p[match['index'], 6].detach().cpu().numpy().astype(np.int64)
        scores = match['confidence'].detach().cpu().numpy()
        matched = np.array([t[t_id, 5].item() if t_id >= 0 else 0 for t_id in match['match']]).astype(np.int64)
        category_id = coco_cat_ids_mapping[c]
        match = {
            'image_id': image_id,
            'category_id': category_id,
            'target_ids': target_ids,
            'pred_ids': pred_ids,
            'matched': matched,
            'scores': scores
        }
        matches.append(match)

In [19]:
print(c_skip)

36


In [12]:
num_points = 101
results = np.full((num_classes, num_points), -1, dtype=np.float32)
for c in range(num_classes):
    if c not in matches_cat:
        print('no class', c)
        continue
    m = matches_cat[c]
    num_targets = sum([rec['num_targets'] for rec in m])
    bbox_m = torch.cat([rec['match'] for rec in m])
    bbox_c = torch.cat([rec['confidence'] for rec in m])

    indices = np.argsort(-bbox_c.detach().cpu().numpy(), kind='mergesort')
    indices = torch.tensor(indices, dtype=torch.int64)
    bbox_c = bbox_c[indices]
    bbox_m = bbox_m[indices]
    
    tp = (bbox_m != -1).cumsum(0)
    fp = (bbox_m == -1).cumsum(0)
    precision = tp / (tp + fp)
    recall = tp / num_targets
    rthresh = torch.linspace(0, 1, num_points)
    iprecision = compute_interpolated_precision(precision, recall, rthresh)
    results[c] = iprecision.detach().cpu().numpy()

In [13]:
results[results > -1].mean()

0.3051732

In [12]:
coco_matches_path = '/media/semyon/Data/Documents/coco/coco_matches.pkl'
with open(coco_matches_path, 'rb') as input_file:
    coco_matches = pickle.load(input_file)
len(coco_matches)

15947

In [20]:
len(matches)

15947

In [13]:
coco_matches = {(m['image_id'], m['category_id']): m for m in coco_matches}
matches = {(m['image_id'], m['category_id']): m for m in matches}

In [14]:
def compare_matches(m_1, m_2):
    if m_1['image_id'] != m_2['image_id']:
        return False, 'image_id'
    if m_1['category_id'] != m_2['category_id']:
        return False, 'category_id'
    t_1 = set(m_1['target_ids'])
    t_2 = set(m_2['target_ids'])
    if t_1 != t_2:
        return False, 'target_ids'
    p_1 = m_1['pred_ids']
    p_2 = m_2['pred_ids']
    if len(p_1) != len(p_2):
        return False, 'pred_ids'
    if not (p_1 == p_2).all():
        return False, 'pred_ids'
    mt_1 = m_1['matched']
    mt_2 = m_2['matched']
    if len(mt_1) != len(mt_2):
        return False, 'matched'
    if not (mt_1 == mt_2).all():
        return False, 'matched'
    s_1 = m_1['scores']
    s_2 = m_2['scores']
    if len(s_1) != len(s_2):
        return False, 'scores'
    if not np.allclose(s_1, s_2):
        return False, 'scores'
    return True, 'ok'

In [15]:
comp = {}
for k in coco_matches:
    if k not in matches:
        comp[k] = 'no entry'
        continue
    s, m = compare_matches(coco_matches[k], matches[k])
    comp[k] = m

In [16]:
set(comp.values())

{'matched', 'ok'}

In [ ]:
num_no = len([v for v in comp.values() if v == 'no entry'])

In [ ]:
print(num_no)
print(len(coco_matches) - len(matches))

In [4]:
v, i = torch.max(torch.rand(5, 80), 1)
i

tensor([44, 61, 67, 18, 46])

In [8]:
os.path.splitext('asdfa')[0]

'asdfa'

In [10]:
len(f'{14:10d}')

10